In [5]:
from bs4 import BeautifulSoup
import requests
import nltk
import math
nltk.download('stopwords')
import pandas as pd
import pickle
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import os
import networkx as nx
import networkx.algorithms.community as nx_community
from nltk.corpus import stopwords
import re
import string
from nltk.tokenize import word_tokenize
import networkx.algorithms.community as nx_comm
import netwulf
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fridajorgensen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Webscraping 
Retrieving all character names from wikisite: 
"https://en.wikipedia.org/wiki/List_of_Harry_Potter_characters"

In [ ]:
# Nedenstående kode outputter noget, men den gør det ud fra tag og alle navne har desværre ikke navne tags :'(
# Så noget andet må gøres...

url = "https://en.wikipedia.org/wiki/List_of_Harry_Potter_characters"
response = requests.get(url)

# Parse the HTML content using Beautiful Soup
soup = BeautifulSoup(response.content, 'html.parser')

# Find all the character names using the tag and class
characters = []
for element in soup.find_all('a', class_='mw-redirect'):
    name = element.text
    # Exclude characters that are not actual character names
    if name.startswith('List') or name.startswith('Category'):
        continue
    characters.append(name)

print(characters)


In [31]:
# bedre, men mange fejl

url = 'https://en.wikipedia.org/wiki/List_of_Harry_Potter_characters'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

character_list = []
content_div = soup.find('div', {'id': 'bodyContent'})
for heading in content_div.find_all('h3'):
    # find the <ul> tag immediately following this heading
    character_ul = heading.find_next_sibling('ul')
    for character_li in character_ul.find_all('li'):
        # extract the character name from the <a> tag within the <li>, if it exists
        character_a = character_li.find('a')
        if character_a:
            character_name = character_a.text
            character_list.append(character_name)

In [30]:
# Ikke optimal løsning, men fikser alle fejlene, så vi kan smide en liste til API'en lidt hurtigt. 
replacements = {
    "Slytherin": "Millicent Bulstrode",
    "Percy Weasley": "Penelope Clearwater",
    "Draco Malfoy": "Astoria Greengrass",
    "Eastern European": "Gregorovitch",
    "Diagon Alley": "Madam Malkin",
    "Horace Slughorn": "Cormac McLaggen",
    "Vanishing Cabinet": "Graham Montague",
    "Harry Potter and the Cursed Child": "Delphi Riddle",
    "Merope Gaunt": "Tom Riddle Sr.",
    "Lord Voldemort": "Thomas Marvolo Riddle",
    "Bogrod": "Gringotts",
    "Gringotts": "Gornuk",
    "Black family": "Kreacher",
    "Death Eater" : "Travers",
    "Barty Crouch Sr": "Winky"
}

for i in range(len(character_list)):
    if character_list[i] in replacements:
        character_list[i] = replacements[character_list[i]]

additions = ["Gabrielle Delacour", 
             "Kendra Dumbledore", 
             "Percival Dumbledore", 
             "Nicolas Flamel", 
             "Griselda Marchbanks", 
             "Eloise Midgen", 
             "Theodore Nott", 
             "Nott Sr", 
             "Albus Severus Potter", 
             "James Sirius Potter", 
             "Lily Luna Potter", 
             "Mary Riddle", 
             "Thomas Riddle", 
             "Demelza Robins", 
             "Salazar Slytherin", 
             "Romilda Vane", 
             "Hugo Weasley", 
             "Rose Weasley",
             "Millicent Bulstrode",
             "Blaise Zabini",
             "Sir Cadogan",
             "Scabior",
             "Trevor"]

print(character_list)


['Hannah Abbott', 'Ludo Bagman', 'Bathilda Bagshot', 'Katie Bell', 'Cuthbert Binns', 'Phineas Nigellus Black', 'Sirius Black', 'Amelia Bones', 'Susan Bones', 'Terry Boot', 'Lavender Brown', 'Millicent Bulstrode', 'Charity Burbage', 'Frank Bryce', 'Alecto Carrow', 'Amycus Carrow', 'Reginald Cattermole', 'Cho Chang', 'Penelope Clearwater', 'Crabbe', 'Vincent Crabbe', 'Colin Creevey', 'Dennis Creevey', 'Dirk Cresswell', 'Winky', 'Barty Crouch Jr', 'John Dawlish', 'Fleur Delacour', 'Dedalus Diggle', 'Amos Diggory', 'Cedric Diggory', 'Elphias Doge', 'Antonin Dolohov', 'Aberforth Dumbledore', 'Albus Dumbledore', 'Ariana Dumbledore', 'Dudley Dursley', 'Marge Dursley', 'Petunia Dursley', 'Vernon Dursley', 'Marietta Edgecombe', 'Arabella Figg', 'Argus Filch', 'Justin Finch-Fletchley', 'Seamus Finnigan', 'Mundungus Fletcher', 'Filius Flitwick', 'Cornelius Fudge', 'Marvolo Gaunt', 'Tom Riddle Sr.', 'Morfin Gaunt', 'Anthony Goldstein', 'Goyle', 'Gregory Goyle', 'Hermione Granger', 'Astoria Greengr